In [ ]:
!pip install livelossplot

In [2]:
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
from keras.layers import *
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!unzip /mydrive/traffic1.zip

In [ ]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    )

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	'traffic/train',
	target_size=(224,224),
	class_mode='categorical',
	batch_size = 64
)

validation_generator = validation_datagen.flow_from_directory(
	'traffic/test',
	target_size=(224,224),
	class_mode='categorical',
	batch_size = 64
)

In [ ]:
train_generator.class_indices

In [7]:
early_stop = EarlyStopping('val_loss', patience=10)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,patience=7, verbose=1)
callbacks = [PlotLossesKerasTF(), early_stop, reduce_lr]

In [8]:
model2 = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
model2.trainable = False
x = model2.output
x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)
resnet50 = Model(inputs=model2.input, outputs=predictions)

94773248/94765736 [==============================] - 1s 0us/step


In [9]:
resnet50.compile(optimizer=Adam(lr=1e-3),loss='mse',metrics=['accuracy'])

In [ ]:
history_resnet50 = resnet50.fit(train_generator, epochs = 60,shuffle = True, validation_data= validation_generator,callbacks=callbacks)

In [14]:
resnet50.save("resnet50.h5")